# Building a baseline model

To tell whether our recommendation system is working well on data or not, we should have a baseline model to compare with. 

* Here, we adopted a popularity model that recommends top-5 most purcahsed product to every users.

- our plan for building a popularity baseline model is like this.
    * 1. build a train matrix with top-5 Product_ID rows and User_ID columns.
    * 2. build a recommendation model with the same matrix frame with the step 1's but with all entries 1.
    * 3. use similarity between two matrices below
        - one with the recommend result matrix, step 2 matrix - step 1 matrix
            * the result matrix would have entry with 1 only when the user didn't buy the product in train data but now they got recommended.
        - the other with the actual result martix with entry only 1 when user bought the product in validation set but not in train set
            * this would act as like a answer.
         - cf. we exclude and keep safe our test data just in case!

#### 1. Load the train and validation data
- These pre-splitted train and validation data is exactly the same with the data set we use to build our own recommendation model.
    * We splitted train and validation of a user who bought more than 100 items.

In [68]:
import pandas as pd
# load the train data
train = pd.read_csv("train_data.csv")

In [63]:
train.head()

,Unnamed: 0,index,User_ID,Product_ID,countProduct,purchased
0,20,151,1000048,P00078742,337,1
215,235,331,1000048,P00261842,337,1
214,234,153,1000048,P00162942,337,1
213,233,274,1000048,P00115142,337,1
212,232,90,1000048,P00214342,337,1


In [5]:
# load the validation data
val = pd.read_csv("val_data.csv")

In [6]:
val.head()

,Unnamed: 0,index,User_ID,Product_ID,countProduct,purchased
0,10,80,1000048,P00103042,337,1
1,11,181,1000048,P00265442,337,1
2,12,212,1000048,P00223142,337,1
3,13,61,1000048,P00147942,337,1
4,14,104,1000048,P00265742,337,1


Since we only need User_ID and Product_ID to build a baseline model, we'll drop all other columns here.

In [7]:
#drop columns of a train data
train = train.drop(['Unnamed: 0', 'index', 'countProduct'], axis=1)
train.head()

,User_ID,Product_ID,purchased
0,1000048,P00078742,1
1,1000048,P00058042,1
2,1000048,P00259342,1
3,1000048,P00052642,1
4,1000048,P00212042,1


In [8]:
#drop columns of a validation data
val = val.drop(['Unnamed: 0', 'index', 'countProduct'], axis=1)
val.head()

,User_ID,Product_ID,purchased
0,1000048,P00103042,1
1,1000048,P00265442,1
2,1000048,P00223142,1
3,1000048,P00147942,1
4,1000048,P00265742,1


In the introduction of building this baseline model, we planned to recommend the top 5 most-frequently pruchased items to all users. This indicates that we don't need any rows with the Product_ID that's not in the list of top-5 Product_ID. For this deletion step, firstly, we need to know the which items are top-5 things.

#### 2. Top-5 most frequently-purchased items
Here, we are going to discover 5 items that were most frequently purchased. To do this, we refer to the code we wrote in the very first of our data pipeline step, exploratory data analysis. Only difference is here we use top-5 but before, we had top-10.

In [10]:
# load an original data set
origin = pd.read_csv("BlackFriday.csv")

In [11]:
#top-5 poducts sold
origin["Product_ID"].value_counts(sort=True)[:5]

P00265242    1858
P00110742    1591
P00025442    1586
P00112142    1539
P00057642    1430
Name: Product_ID, dtype: int64

Now, it's time to delete all the rows with non-top-5 items!

In [12]:
new_train_1 = train[(train.Product_ID == 'P00265242')]
new_train_2 = train[(train.Product_ID == 'P00110742')]
new_train_3 = train[(train.Product_ID == 'P00025442')]
new_train_4 = train[(train.Product_ID == 'P00112142')]
new_train_5 = train[(train.Product_ID == 'P00057642')]

In [13]:
new_train_1.head()

,User_ID,Product_ID,purchased
27,1000048,P00265242,1
965,1000123,P00265242,1
1831,1000169,P00265242,1
2370,1000195,P00265242,1
2952,1000202,P00265242,1


In [14]:
new_train_2.head()

,User_ID,Product_ID,purchased
49,1000048,P00110742,1
606,1000053,P00110742,1
718,1000123,P00110742,1
1563,1000149,P00110742,1
4297,1000308,P00110742,1


In [15]:
new_train = pd.concat([new_train_1, new_train_2, new_train_3, new_train_4, new_train_5])

In [18]:
new_train.head()

,User_ID,Product_ID,purchased
27,1000048,P00265242,1
965,1000123,P00265242,1
1831,1000169,P00265242,1
2370,1000195,P00265242,1
2952,1000202,P00265242,1


In [20]:
new_val_1 = val[(val.Product_ID == 'P00265242')]
new_val_2 = val[(val.Product_ID == 'P00110742')]
new_val_3 = val[(val.Product_ID == 'P00025442')]
new_val_4 = val[(val.Product_ID == 'P00112142')]
new_val_5 = val[(val.Product_ID == 'P00057642')]

In [21]:
new_val = pd.concat([new_val_1, new_val_2, new_val_3, new_val_4, new_val_5])

In [22]:
new_val.head()

,User_ID,Product_ID,purchased
611,1001242,P00265242,1
682,1001303,P00265242,1
1406,1002453,P00265242,1
1838,1003476,P00265242,1
1877,1003519,P00265242,1


In [38]:
train = pd.pivot_table(new_train, values='purchased', index='User_ID', columns='Product_ID', fill_value=0)

In [40]:
train.head()

Product_ID,P00025442,P00057642,P00110742,P00112142,P00265242
User_ID,,,,,
1000048,0,0,1,1,1
1000053,0,1,1,1,0
1000123,0,1,1,0,1
1000148,0,1,0,1,0
1000149,0,0,1,1,0


In [41]:
val = pd.pivot_table(new_val, values='purchased', index='User_ID', columns='Product_ID', fill_value=0)

In [42]:
val.head()

Product_ID,P00025442,P00057642,P00110742,P00112142,P00265242
User_ID,,,,,
1000329,0,1,0,0,0
1001068,0,0,0,1,0
1001224,0,0,1,0,0
1001242,0,0,0,0,1
1001266,0,0,1,0,0


In [43]:
recom = pd.pivot_table(new_train, index='User_ID', columns='Product_ID', fill_value=1)

In [44]:
recom.head()

purchased                                        
Product_ID P00025442 P00057642 P00110742 P00112142 P00265242
User_ID                                                     
1000048            1         1         1         1         1
1000053            1         1         1         1         1
1000123            1         1         1         1         1
1000148            1         1         1         1         1
1000149            1         1         1         1         1

In [45]:
recom = recom - train

In [47]:
recom.head()

purchased                                        
Product_ID P00025442 P00057642 P00110742 P00112142 P00265242
User_ID                                                     
1000048            1         1         0         0         0
1000053            1         0         0         0         1
1000123            1         0         0         1         0
1000148            1         0         1         0         1
1000149            1         1         0         0         1

In [69]:
users = pd.DataFrame(train.User_ID.unique())
users.columns = ['User_ID']
users_length = len(users)

top5products = pd.DataFrame(new_val.Product_ID.unique())
top5products.columns = ['Product_ID']
top5products_length = len(top5products)

In [70]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A,B)/(norm(A)*norm(B))

In [71]:
recom.loc[1000048]

           Product_ID
purchased  P00025442     1
           P00057642     1
           P00110742     0
           P00112142     0
           P00265242     0
Name: 1000048, dtype: int64

In [101]:
type(val.index)

pandas.core.indexes.numeric.Int64Index

In [106]:
val_user_info = pd.DataFrame(val.index)
val_user_info.columns = ['User_ID']

In [103]:
val_user_info

,User_ID
0,1000329
1,1001068
2,1001224
3,1001242
4,1001266
5,1001303
6,1001599
7,1001605
8,1001889
9,1002092


In [107]:

val_users_length = len(val_user_info)
similarity_matrix = np.zeros(shape=(val_users_length, 1))


for i, user1 in enumerate(val_user_info['User_ID']):
    for user2 in recom.index:
        if(user1 == user2):
            user_recom = np.array(recom.loc[user1])
            user_val = np.array(val.loc[user2])
    
            similarity_matrix[i] = cos_sim(user_recom, user_val)
            
            print(i, user1)

0 1000329
2 1001224
3 1001242
4 1001266
5 1001303
6 1001599
7 1001605
8 1001889
9 1002092
10 1002109
11 1002453
12 1003391
13 1003476
14 1003507
15 1003519
16 1003648
17 1003683
18 1003693
19 1004028
20 1005111
21 1005306
22 1005312


In [108]:
similarity_matrix

array([[0.57735027],
       [0.        ],
       [0.5       ],
       [0.5       ],
       [0.57735027],
       [0.57735027],
       [1.        ],
       [0.5       ],
       [0.57735027],
       [0.5       ],
       [0.70710678],
       [0.57735027],
       [0.57735027],
       [0.70710678],
       [0.57735027],
       [0.5       ],
       [0.5       ],
       [0.70710678],
       [0.70710678],
       [0.5       ],
       [0.5       ],
       [0.57735027],
       [0.57735027]])

In [77]:
val['User_ID'].count_values()

KeyError: 'User_ID'